In [1]:
import sklearn
import pandas as pd
import numpy as np
import tensorflow as tf
import keras
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


In [2]:
train_file = pd.read_csv('all/train.tsv',sep='\t')

In [3]:
words = []
word_vectors = []
word_to_indx = {}
indx = 0
with open('glove.6B/glove.6B.50d.txt',encoding='utf-8') as file:
    for indx,line in enumerate(file):
        l = line.split()
        word = l[0]
        words.append(word)
        word_to_indx[word] = indx+1
        word_vec = np.asarray(l[1:],dtype='float32')
        word_vectors.append(word_vec)

word_to_indx['<PAD>'] = 0
word_vectors.insert(0,np.zeros(word_vec.shape))
print(len(word_vectors))
print(word_vectors[0].shape)
print(word_vec.shape[0])

400001
(50,)
50


In [4]:
word_vectors_new = np.vstack(word_vectors)
#word_vectors_new.shape

In [5]:
def get_word_embedding(word):
    if word in word_to_indx:
        return word_vectors_new[word_to_indx[word]]
    else:
        return word_vectors_new[0]       

In [6]:
from keras.preprocessing.text import text_to_word_sequence

def get_seq_embedding(sentence):
    tokens = text_to_word_sequence(sentence)
    embeddings = []
    for i in tokens:
        embeddings.append(get_word_embedding(i.lower()))
    #for i in range(0,max_len-len(tokens)):
    #    embeddings.append(get_word_embedding('<PAD>'))
    return embeddings

In [7]:
train_file_selected = train_file[['Phrase','Sentiment']]
#x_train = [get_seq_embedding(i) for i in train]
#y_train = np.asarray(train['Sentiment'])

In [8]:
msk = np.random.rand(len(train_file_selected)) < 0.8
train = train_file_selected[msk]
test = train_file_selected[~msk]
train.reset_index(drop=True,inplace=True)
test.reset_index(drop=True,inplace=True)

In [9]:
len(train), len(test), len(train_file_selected)

(124914, 31146, 156060)

In [10]:
# batch train data generator
'''
temp = x[:10]
tf.convert_to_tensor(temp)
batched_data = tf.train.batch(tensors=x,batch_size=5,dynamic_pad=True)
'''

def generate_train_batch(batch_size):
    sample_indices = np.random.choice(train.shape[0],size=batch_size,replace=False)
    x = []
    y = []
    for i in sample_indices:
        x.append(get_seq_embedding(train['Phrase'][i]))
        y.append(train['Sentiment'][i])
    return pad_sequences(x,dtype='float32'),y

In [11]:
# batch test data generator

def generate_test_batch(batch_size):
    sample_indices = np.random.choice(test.shape[0],size=batch_size,replace=False)
    x = []
    y = []
    for i in sample_indices:
        x.append(get_seq_embedding(test['Phrase'][i]))
        y.append(test['Sentiment'][i])
    return pad_sequences(x,dtype='float32'),y

In [12]:
# model
'''
embedding_size = 100
learning_rate = 0.0001

graph_input = tf.placeholder(shape=(None,None,embedding_size),dtype=tf.float32)
graph_output = tf.placeholder(shape=(None),dtype=tf.int64)
graph_training = tf.placeholder(shape=(), dtype=tf.bool)

#rnn_cell = tf.keras.layers.SimpleRNNCell(units=128)
rnn_cell = tf.nn.rnn_cell.LSTMCell(128)
rnn_out,rnn_state = tf.nn.dynamic_rnn(cell=rnn_cell,inputs=graph_input,dtype=tf.float32)
dense = tf.layers.dense(inputs=rnn_out[:,-1,:],units=256,activation=tf.nn.relu)
dense = tf.layers.dense(inputs=dense,units=128,activation=tf.nn.relu)
dropout = =tf.layers.dropout
dense = tf.layers.dense(inputs=dense,units=64,activation=tf.nn.relu)
logits = tf.layers.dense(inputs=dense,units=5,activation=tf.nn.relu)
probs = tf.nn.softmax(logits)
labels = tf.argmax(probs,axis=1)
#print(rnn_out.get_shape(),dense.get_shape())
#print(logits.get_shape(),probs.get_shape(),labels.get_shape())
cross_entropy_loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits,labels=graph_output))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy_loss)
correct_labels = tf.equal(labels,graph_output)
accuracy = tf.reduce_mean(tf.cast(correct_labels,tf.float32))
'''

'\nembedding_size = 100\nlearning_rate = 0.0001\n\ngraph_input = tf.placeholder(shape=(None,None,embedding_size),dtype=tf.float32)\ngraph_output = tf.placeholder(shape=(None),dtype=tf.int64)\ngraph_training = tf.placeholder(shape=(), dtype=tf.bool)\n\n#rnn_cell = tf.keras.layers.SimpleRNNCell(units=128)\nrnn_cell = tf.nn.rnn_cell.LSTMCell(128)\nrnn_out,rnn_state = tf.nn.dynamic_rnn(cell=rnn_cell,inputs=graph_input,dtype=tf.float32)\ndense = tf.layers.dense(inputs=rnn_out[:,-1,:],units=256,activation=tf.nn.relu)\ndense = tf.layers.dense(inputs=dense,units=128,activation=tf.nn.relu)\ndropout = =tf.layers.dropout\ndense = tf.layers.dense(inputs=dense,units=64,activation=tf.nn.relu)\nlogits = tf.layers.dense(inputs=dense,units=5,activation=tf.nn.relu)\nprobs = tf.nn.softmax(logits)\nlabels = tf.argmax(probs,axis=1)\n#print(rnn_out.get_shape(),dense.get_shape())\n#print(logits.get_shape(),probs.get_shape(),labels.get_shape())\ncross_entropy_loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_e

In [13]:
class Model:
    
    def __init__(self,is_training,embedding_size,learning_rate,dropout_rate):
        self.is_training = is_training
        self.embedding_size = embedding_size
        self.learning_rate = learning_rate
        self.dropout_rate = dropout_rate
        
    def build(self):
        self.graph_input = tf.placeholder(shape=(None,None,self.embedding_size),dtype=tf.float32)
        self.graph_output = tf.placeholder(shape=(None),dtype=tf.int64)
        self.graph_training = tf.placeholder(shape=(), dtype=tf.bool)
        
        rnn_cell = tf.nn.rnn_cell.LSTMCell(10)
        rnn_out,rnn_state = tf.nn.dynamic_rnn(cell=rnn_cell,inputs=self.graph_input,dtype=tf.float32)
        dense = tf.layers.dense(inputs=rnn_out[:,-1,:],units=256,activation=tf.nn.relu)
        dropout = tf.layers.dropout(inputs=dense,rate=self.dropout_rate,training=self.is_training)
        dense = tf.layers.dense(inputs=dropout,units=1200,activation=tf.nn.relu)
        self.logits = tf.layers.dense(inputs=dense,units=5,activation=tf.nn.relu)
        self.probs = tf.nn.softmax(self.logits)
        self.labels = tf.argmax(self.probs,axis=1)
        #print(rnn_out.get_shape(),dense.get_shape())
        #print(logits.get_shape(),probs.get_shape(),labels.get_shape())
        self.cross_entropy_loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=self.logits,labels=self.graph_output))
        self.optimizer = tf.train.AdamOptimizer(self.learning_rate).minimize(self.cross_entropy_loss)
        self.correct_labels = tf.equal(self.labels,self.graph_output)
        self.accuracy = tf.reduce_mean(tf.cast(self.correct_labels,tf.float32))
        

In [14]:
bestModelPath = '/Users/m0k00ew/Desktop/MLDL/kaggle/models_saved/lstme60b32lr0001dout6emb50/bestModel.ckpt'

In [15]:
# training

model = Model(is_training=True,embedding_size=50,learning_rate=0.001,dropout_rate=0.6)
model.build()

epochs = 60 #100
batch_size = 32
best_test_acc = None
sess = tf.Session()
init = tf.initializers.global_variables()
sess.run(init)
sess.run(tf.local_variables_initializer())

for epoch in range(0,epochs):
    sess.run(tf.local_variables_initializer())
    train_steps_per_epoch = train.shape[0]//batch_size
    test_steps_per_epoch = test.shape[0]//batch_size
    train_losses = []
    train_accs = []
    test_losses = []
    test_accs = []
    
    saver = tf.train.Saver()
    
    for train_step in range(0,train_steps_per_epoch):
        x,y = generate_train_batch(batch_size)
        #print(x.shape,len(y))
        #rnn_o,dense_o,logits_o,probs_o,labels_o = sess.run([rnn_out,dense,logits,probs,labels],feed_dict={graph_input:x,graph_output:y})
        #print(rnn_o.shape,logits_o.shape,probs_o.shape,labels_o.shape)
        #_ = sess.run(optimizer,feed_dict={graph_input:x,graph_output:y})
        cross_ent,_,correct_lab,acc = sess.run([model.cross_entropy_loss,model.optimizer,model.correct_labels,model.accuracy],feed_dict={model.graph_input:x,model.graph_output:y})
        train_losses.append(cross_ent)
        train_accs.append(acc)
    
    for test_step in range(0,test_steps_per_epoch):
        x,y = generate_test_batch(batch_size)
        cross_ent_test,correct_lab_test,acc_test = sess.run([model.cross_entropy_loss,model.correct_labels,model.accuracy],feed_dict={model.graph_input:x,model.graph_output:y})
        test_losses.append(cross_ent_test)
        test_accs.append(acc_test)
    if best_test_acc is None or np.mean(test_accs) >= best_test_acc:
        best_test_acc = np.mean(test_accs)
        save_path = saver.save(sess, bestModelPath)
        print("Model saved in path: %s" % save_path)

    print ("epoch",epoch)
    print ("train loss",np.mean(train_losses))
    print ("train acc",np.mean(train_accs))
    print ("test loss",np.mean(test_losses))
    print ("test acc",np.mean(test_accs))
    print ("\n")
sess.close()


Model saved in path: /Users/m0k00ew/Desktop/MLDL/kaggle/models_saved/lstme60b32lr0001dout6emb50/bestModel.ckpt
epoch 0
train loss 1.1796503
train acc 0.546935
test loss 1.1481545
test acc 0.5648767


epoch 1
train loss 1.1414216
train acc 0.5635969
test loss 1.1411219
test acc 0.56391317


Model saved in path: /Users/m0k00ew/Desktop/MLDL/kaggle/models_saved/lstme60b32lr0001dout6emb50/bestModel.ckpt
epoch 2
train loss 1.1261456
train acc 0.56916153
test loss 1.1141078
test acc 0.5754432


epoch 3
train loss 1.1297284
train acc 0.5666394
test loss 1.1215314
test acc 0.5747045


epoch 4
train loss 1.1190493
train acc 0.56964195
test loss 1.1332738
test acc 0.56696427


epoch 5
train loss 1.114189
train acc 0.57266045
test loss 1.1337478
test acc 0.56561536


epoch 6
train loss 1.1155199
train acc 0.5698501
test loss 1.1204203
test acc 0.57377315


epoch 7
train loss 1.1092359
train acc 0.57412565
test loss 1.120158
test acc 0.57287383


epoch 8
train loss 1.1094661
train acc 0.575719
test

In [16]:
test_file = pd.read_csv('all/test.tsv',sep='\t')

In [17]:
test = test_file['Phrase']

In [18]:
test_file

,PhraseId,SentenceId,Phrase
0,156061,8545,An intermittently pleasing but mostly routine ...
1,156062,8545,An intermittently pleasing but mostly routine ...
2,156063,8545,An
3,156064,8545,intermittently pleasing but mostly routine effort
4,156065,8545,intermittently pleasing but mostly routine
5,156066,8545,intermittently pleasing but
6,156067,8545,intermittently pleasing
7,156068,8545,intermittently
8,156069,8545,pleasing
9,156070,8545,but


In [19]:
x_test = [get_seq_embedding(i) for i in test]

In [20]:
len(x_test)

66292

In [21]:
x_test = pad_sequences(x_test,dtype='float32')

In [22]:

tf.reset_default_graph()

model = Model(is_training=False,embedding_size=50,learning_rate=0,dropout_rate=0)
model.build()
saver = tf.train.Saver()
sess = tf.Session()



saver.restore(sess, bestModelPath)

init = tf.initializers.global_variables()
sess.run(init)
sess.run(tf.local_variables_initializer())

test_probs,test_labels = sess.run([model.probs,model.labels],feed_dict={model.graph_input:x_test})

sess.close()

INFO:tensorflow:Restoring parameters from /Users/m0k00ew/Desktop/MLDL/kaggle/models_saved/lstme60b32lr0001dout6emb50/bestModel.ckpt


In [23]:
test_labels

array([4, 4, 4, ..., 4, 4, 4])

In [24]:
submit = pd.DataFrame(data={'PhraseId':test_file['PhraseId'],'Sentiment':test_labels})

In [25]:
submit.to_csv('submit_lstm_epoch60_batch32_0001_dout6_emb50.csv',index=False)